In [4]:
# Map of US Oil Refineries
import pandas as pd

In [59]:
# Read in Refinery Data
Refineries = pd.read_excel('refcap18.xls')
Refineries.head()

,CORPORATION,SURVEY,PERIOD,COMPANY_NAME,RDIST_LABEL,STATE_NAME,SITE,PADD,PRODUCT,SUPPLY,QUANTITY
0,DELEK GROUP LTD,820,18,ALON REFINING KROTZ SPRINGS INC,Louisiana Gulf Coast,Louisiana,KROTZ SPRINGS,3,CAT CRACKING: FRESH FEED,"Downstream Charge Capacity, Current Year (barr...",33000
1,DELEK GROUP LTD,820,18,ALON REFINING KROTZ SPRINGS INC,Louisiana Gulf Coast,Louisiana,KROTZ SPRINGS,3,CAT CRACKING: FRESH FEED,"Downstream Charge Capacity, Current Year (barr...",34000
2,DELEK GROUP LTD,820,18,ALON REFINING KROTZ SPRINGS INC,Louisiana Gulf Coast,Louisiana,KROTZ SPRINGS,3,CAT CRACKING: FRESH FEED,"Downstream Charge Capacity, Next Year (barrels...",34000
3,DELEK GROUP LTD,820,18,ALON REFINING KROTZ SPRINGS INC,Louisiana Gulf Coast,Louisiana,KROTZ SPRINGS,3,CAT REFORMING: HIGH PRESSURE,"Downstream Charge Capacity, Current Year (barr...",12000
4,DELEK GROUP LTD,820,18,ALON REFINING KROTZ SPRINGS INC,Louisiana Gulf Coast,Louisiana,KROTZ SPRINGS,3,CAT REFORMING: HIGH PRESSURE,"Downstream Charge Capacity, Current Year (barr...",13000


In [60]:
# Create a column to search google API by (Corp + Site + State)
Refineries['Refinery'] = Refineries['CORPORATION'] + ' ' + Refineries['SITE'] + ' ' + Refineries['STATE_NAME']

In [68]:
# Group Refineries by each google search term (unique refinery), and sum the quantities
Grouped = Refineries.groupby('Refinery')
SumTotal = pd.DataFrame(Grouped['QUANTITY'].agg('sum'))
SumTotal.head()

,QUANTITY
Refinery,
ACCESS INDUSTRIES CHANNELVIEW Texas,44000
ACCESS INDUSTRIES HOUSTON Texas,3326112
AMERICAN REFINING GROUP INC BRADFORD Pennsylvania,76820
ANDEAVOR ANACORTES Washington,1242608
ANDEAVOR CARSON California,2904162


In [64]:
# Create google API query to pull location data based on geocords
import requests

Data = {}
for refinery in list(SumTotal.index):
    response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?key=ABCDEFG&address=' + refinery)

    resp_json_payload = response.json()
    if resp_json_payload['status'] != 'ZERO_RESULTS':
        Data[refinery] = resp_json_payload['results'][0]['geometry']['location']

{'lat': 29.7760599, 'lng': -95.11465330000001}
{'lat': 40.7628678, 'lng': -73.9745762}
{'lat': 41.9662859, 'lng': -78.628391}
{'lat': 48.4963294, 'lng': -122.5667318}
{'lat': 33.8228302, 'lng': -118.2430538}
{'lat': 46.8781185, 'lng': -102.7681903}
{'lat': 31.7700948, 'lng': -106.4130854}
{'lat': 35.4869223, 'lng': -108.4275321}
{'lat': 60.6865684, 'lng': -151.3695127}
{'lat': 46.85422759999999, 'lng': -100.886846}
{'lat': 38.0239143, 'lng': -122.0634835}
{'lat': 44.85280100000001, 'lng': -92.99727899999999}
{'lat': 40.7891907, 'lng': -111.9037248}
{'lat': 33.7919627, 'lng': -118.2341233}
{'lat': 64.7511111, 'lng': -147.3494443}
{'lat': 61.13083330000001, 'lng': -146.3483334}
{'lat': 29.26155, 'lng': -97.784657}
{'lat': 41.674344, 'lng': -83.45742899999999}
{'lat': 48.891652, 'lng': -122.601045}
{'lat': 70.41049579999999, 'lng': -148.9721164}
{'lat': 41.6624674, 'lng': -87.4832644}
{'lat': 27.8181422, 'lng': -97.5033539}
{'lat': 32.7973978, 'lng': -93.41480580000001}
{'lat': 47.522787,

In [65]:
# Combine lat and lon data with quantity for visualization
Geo_Coords = pd.DataFrame(Data).transpose()
Geo_Coords.head()
Final = Geo_Coords.join(SumTotal)
Final.to_csv('Refineries_US.csv')

,lat,lng
ACCESS INDUSTRIES CHANNELVIEW Texas,29.776060,-95.114653
ACCESS INDUSTRIES HOUSTON Texas,40.762868,-73.974576
AMERICAN REFINING GROUP INC BRADFORD Pennsylvania,41.966286,-78.628391
ANDEAVOR ANACORTES Washington,48.496329,-122.566732
ANDEAVOR CARSON California,33.822830,-118.243054
